# Определение токсичности комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. 

Требуется инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Комментарии на английском языке.

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. 

Заказчиком предоставлен набор данных с разметкой о токсичности правок.

Заказчик предоставли требование о метрике качества F1 не меньше 0.75. 

## Описание данных

Данные находятся в файле `toxic_comments.csv`. 

* *text*  - содержит текст комментария
* *toxic* - целевой признак

## План работ

0. Импорт библиотек

1. Загрузка и подготовка данных данных

    1.1. Загрузка данных
    
    1.2. Исследование данных
    
    1.3. Дисбаланс классов
    
    1.4. Очистка и лемматизация
    
    1.5. Подготовка признаков
    
2. Обучение моделей

3. Тестирование лучшей модели

4. Выводы    

## 0. Импорт библиотек

In [1]:
import pandas as pd
import re
import nltk
import spacy
import numpy as np
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline
from time import time
pd.set_option('max_colwidth', 100)

## 1. Загрузка и подготовка данных

### 1.1. Загрузка данных

Загрузим данные и посмотрим на часть данных

In [2]:
df = pd.read_csv("datasets/toxic_comments.csv")
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


### 1.2. Исследование данных

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Пропусков нет - хорошо.

Изменим тип данных toxic для экономии памяти

In [4]:
df.toxic = df.toxic.astype('uint8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  uint8 
dtypes: int64(1), object(1), uint8(1)
memory usage: 2.6+ MB


Сократили использование памяти в 1.38 раз - память нам пригодится.

### 1.3. Дисбаланс классов

In [5]:
toxic_total = df['toxic'].sum()
non_toxic_total = df.shape[0] - toxic_total
print(f'Количество строк с токсичными комментариями: {toxic_total} ({toxic_total/df.shape[0]:.2%})')
print(f'Количество строк с нетоксичными комментариями: {non_toxic_total} ({non_toxic_total/df.shape[0]:.2%})')

Количество строк с токсичными комментариями: 16186 (10.16%)
Количество строк с нетоксичными комментариями: 143106 (89.84%)


Наблюдается существенный дисбаланс классов (1 к 9). Учтем это при построении моделей.

### 1.4. Очистка и лемматизация

Очистим и лемматизируем текст

In [6]:
# для ускорения работы учебного проекта возьмем часть данных
df = df.sample(80000, random_state=12345)

# очистим текст от символов, пунктуации, спецсимволов и т.п.
def clear_text(text):
     return ' '.join(re.sub(r'[^a-zA-Z ]', ' ', text).split())
    
df['text'] = df['text'].apply(lambda x: clear_text(x))

df[['text']].head()

,text
109486,Expert Categorizers Why is there no mention of the fact that Nazis were particularly great categ...
104980,Noise fart talk
82166,An indefinite block is appropriate even for a minor infraction if you show know signs of discont...
18721,I don t understand why we have a screenshot of AP s GUI but not UB Can someone remedy this
128178,Hello Some of the people places or things you have written about in the article Nikolas Tryfonos...


Очистка прошла успешно

Лемматизируем текст

In [7]:
%%time

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

lemm_texts = []
nlp_pipe = nlp.pipe(df['text'].values, disable = ['ner', 'parser'])

for doc in nlp_pipe:
    lemm_text = " ".join([token.lemma_ for token in doc])
    lemm_texts.append(lemm_text)

df['text'] = lemm_texts
df[['text']].head()

Wall time: 18min 20s


,text
109486,Expert categorizer why be there no mention of the fact that Nazis be particularly great categori...
104980,noise fart talk
82166,an indefinite block be appropriate even for a minor infraction if you show know sign of disconti...
18721,I don t understand why we have a screenshot of AP s GUI but not UB can someone remedy this
128178,hello some of the people place or thing you have write about in the article Nikolas Tryfonos may...


Лемматизация прошла успешно

## 1.5. Подготовка признаков

In [8]:
X = df['text'].values
y = df['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape)
print("y_test.shape", y_test.shape)

X_train.shape (72000,)
y_train.shape (72000,)
X_test.shape (8000,)
y_test.shape (8000,)


In [9]:
nltk.download('stopwords')
stopwords = nltk_stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2. Обучение

In [10]:
#dictionary pipelines of classifiers and vectorizers for SearchCV
pipelines = {}

pipelines['LogisticRegression'] = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("clf", LogisticRegression()),
    ]
)

pipelines['DecisionTreeClassifier'] = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("clf", DecisionTreeClassifier())
    ]
)

pipelines['SGDClassifier'] = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("clf", SGDClassifier())
    ]
)

#dictionary of parameters of classifiers and vectorizers for SearchCV
params = {}
params['LogisticRegression']={
    "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5, 10),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    "vect__norm": ("l1", "l2"),
    'clf__max_iter' : [5000],
    'clf__random_state' : [12345],
    'clf__class_weight' : ['balanced'],
    'clf__C' : list(np.arange(1.0, 16.0, 0.5)),
}

params['DecisionTreeClassifier']={
    "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5, 10),
    "vect__norm": ("l1", "l2"),
    'clf__random_state' : [12345],
    'clf__class_weight' : ['balanced'],
    'clf__max_depth': list(range(6, 13, 2)),
    'clf__min_samples_split' :  list(range(2, 13, 2)),
    'clf__min_samples_leaf' : list(range(1, 13, 2)),
}

params['SGDClassifier']={
    "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5, 10),
    "vect__norm": ("l1", "l2"),
    'clf__loss': ['hinge'],
    'clf__penalty': ['l2'],
    'clf__alpha': np.logspace(-6, 6, 13),
    'clf__max_iter': [5],
    'clf__class_weight' : ['balanced'],
    'clf__tol': [None],
    'clf__random_state' : [12345],
}

# dictionary for saving results
results = dict()


In [11]:
%%time

for i in pipelines.keys():
    rs = RandomizedSearchCV(
        estimator=pipelines[i],
        param_distributions=params[i],
        n_iter=40,
        random_state=12345,
        n_jobs=2,
        verbose=1,
        scoring='f1',
        cv=3
    )

    rs.fit(X_train, y_train)

    #saving results
    results[i] = dict()
    results[i]['train f1'] = rs.best_score_
    results[i]['best_params_'] = rs.best_params_
    results[i]['best_estimator_'] = rs.best_estimator_
    results[i]['mean_fit_time'] = rs.cv_results_['mean_fit_time'][rs.best_index_]


    # printing results
    print(i)
    print('train f1:', results[i]['train f1'])
    print('best_params_:', results[i]['best_params_'])
    print('mean_fit_time:', results[i]['mean_fit_time'])
    print('_______________')
    print()

Fitting 3 folds for each of 40 candidates, totalling 120 fits
LogisticRegression
train f1: 0.7732490510712992
best_params_: {'vect__norm': 'l2', 'vect__ngram_range': (1, 2), 'vect__min_df': 1, 'vect__max_df': 0.6, 'clf__random_state': 12345, 'clf__max_iter': 5000, 'clf__class_weight': 'balanced', 'clf__C': 15.5}
mean_fit_time: 86.59038472175598
_______________

Fitting 3 folds for each of 40 candidates, totalling 120 fits
DecisionTreeClassifier
train f1: 0.5863740419922725
best_params_: {'vect__norm': 'l2', 'vect__min_df': 3, 'vect__max_df': 0.2, 'clf__random_state': 12345, 'clf__min_samples_split': 10, 'clf__min_samples_leaf': 1, 'clf__max_depth': 12, 'clf__class_weight': 'balanced'}
mean_fit_time: 51.451926390329994
_______________

Fitting 3 folds for each of 40 candidates, totalling 120 fits
SGDClassifier
train f1: 0.7292202526014622
best_params_: {'vect__norm': 'l2', 'vect__min_df': 3, 'vect__max_df': 1.0, 'clf__tol': None, 'clf__random_state': 12345, 'clf__penalty': 'l2', 'clf__m

**Лучшая модель: LogisticRegression**

c параметрами:
TfidfVectorizer (norm='l2', ngram_range=(1, 2), min_df=1, max_df=0.6)

LogisticRegression(random_state=12345, max_iter=5000, class_weight='balanced', C=15.5}

train f1: 0.7732490510712992

## 3. Тестирование лучшей модели

In [12]:
i = 'LogisticRegression'
results[i]['test f1'] = f1_score(y_test, results[i]['best_estimator_'].predict(X_test))
print(i)
print('train f1:', results[i]['train f1'])
print('test f1:', results[i]['test f1'])


LogisticRegression
train f1: 0.7732490510712992
test f1: 0.7936507936507937


## 4. Выводы

Текст очищен и лемматизирован.

Имеется дисбаланс классов (токсичных комментариев околло 10%).

проведен подбор гиперпараметров векторизации и нескольких моделей в пайплайне:
- LogisticRegression
- DecisionTreeClassifier
- SGDClassifier

Выбрана лучшая модель: **LogisticRegression**

c параметрами:
TfidfVectorizer (norm='l2', ngram_range=(1, 2), min_df=1, max_df=0.6)

LogisticRegression(random_state=12345, max_iter=5000, class_weight='balanced', C=15.5)

train f1:  0.7732

test f1 = 0.7937